In [ ]:
!pip install plotly
!pip install BioPython
!pip install Bio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import IPython.display as ipd
import plotly.graph_objects as go
from Bio.Seq import Seq
from Bio import Entrez, SeqIO
from Bio.SeqUtils import molecular_weight
from Bio.SeqUtils import MeltingTemp as mt



class DNASequence:
    def __init__(self, sequence, label='DNA Sequence'):
        """Function initializes the DNASequence object with sequence and label"""
        self.sequence = sequence.upper() #Convert sequence to uppercase
        self.label = label #Label for the sequence
        self.start_codons = ['ATG', 'TTG', 'CTG', 'GTG'] #List of start codons
        self.stop_codons = ['TAA', 'TAG', 'TGA'] #List of stop codons


    def get_gc_content(self, subseq=None):
      ''' Function computes GC content of input sequence'''
      sequence = subseq if subseq is not None else self.sequence  #Use subsequence if provided
      gc_count = sum(1 for base in sequence if base in "GC") #Count G and C bases
      total_bases = sum(1 for base in sequence if base in 'ATCG') #Count all valid base
      return (gc_count / total_bases) * 100 if total_bases > 0 else 0 #Calculate GC content percentage


    def orf_seqs(self):
        ''' Function computes list of open reading frames of input sequence'''
        orfs = [] #Initialize list to store ORFs
        for i in range(len(self.sequence)):
          if self.sequence[i:i+3] in self.start_codons: #Check for start codon
            for j in range(i+3, len(self.sequence), 3):
              if self.sequence[j:j+3] in self.stop_codons: #Check for stop codon
                 orfs.append(self.sequence[i:j+3]) #Append ORF to list
        return orfs

    def transcription(self):
        ''' Function computes the transcript of input sequence'''
        complement = {'A': 'U', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'} #DNA to RNA complement
        return ''.join(complement.get(base, 'N') for base in self.sequence) #Transcribe to RNA

    def id_palindromes(self):
        """Function identifies palindromic sequences in the DNA sequence"""
        pal = [] #Initialize list to store palindromes
        for i in range(len(self.sequence) - 1):
         for j in range(i+4, len(self.sequence)):
          if self.sequence[i:j] == self.sequence[j-1:i-1:-1]: #Check if sequence is palindrome
            pal.append(self.sequence[i:j])
        return pal

    def count_kmers(self, k):
      ''' Function computes kmers of specified length of input sequence'''
      kmers = {} #Initialize dictionary to store k-mer counts
      for i in range(len(self.sequence) - k + 1):
        kmer = self.sequence[i:i+k] #Extract k-mer
        kmers[kmer] = kmers.get(kmer, 0) + 1 #Count k-mer occurrences
      return kmers

    def melting_temperature(self):
        ''' Function computes melting temperature of input sequence'''
        gc_cont = (self.sequence.count("G") + self.sequence.count("C"))/len(self.sequence) #Calculate GC content
        melt_temp = (64.9 + 41 * (gc_cont - 16.4) )/len(self.sequence) #Calculate melting temperature
        return melt_temp


    def design_primers(self, length=20, min_tm=52, max_tm=58, min_gc=40, max_gc=60):
      ''' Function returns a list of primers of length 20 from the input sequence'''
      init_primers = [] #Initialize list to store primer candidates
      for i in range(len(self.sequence) - length + 1):
       seq = self.sequence[i:i+length] #Extract potential primer
       gc_content = self.get_gc_content(seq) #Calculate GC content
       tm = mt.Tm_Wallace(seq) #Calculate melting temperature
       if min_tm <= tm <= max_tm and min_gc <= gc_content <= max_gc: #Check constraints
          init_primers.append(seq) #Append valid primer to list
       return init_primers

    def has_hairpin(self,seq):
      ''' Function check if there are haiprin structures in input sequence'''
      comp = 0
      not_comp = 0
      for i in range(10):
        to_check_seq_1 = seq[i] #Base at position i from the start of the sequence
        to_check_seq_2 = seq[-i-1] #Base at position i from the end of the sequence
        if to_check_seq_1 == ComplementarySequence(to_check_seq_2).get_reverse_complement():
          if not_comp > 0:
            return False
          comp += 1  #Pair
        else:
            not_comp += 1
      if comp > 0 and 2 <= not_comp <= 4:
       return True
      return False

    def filter_primers(self, primers):
      ''' Function filters out primers without any hairpin '''
      return [p for p in primers if not self.has_hairpin(p)]

    def save_results(self):
      ''' Function opens a single file and writes all the results to it'''
      #Open a single file to write all results
      with open(f"{self.label}_all_results.txt", 'w') as f:
        f.write(f'Accession Number: {self.label}\n')
        f.write(f"GC%: {self.get_gc_content():.2f}\n")
        f.write(f"Transcript: {self.transcription()}\n")

        #Complementary and Reverse Complementary Sequences
        comp_seq = ComplementarySequence(self.sequence).get_complementary_sequence()
        rev_comp_seq = ComplementarySequence(self.sequence).get_reverse_complement()
        f.write(f"Double stranded DNA\n")
        f.write(f"5' {self.sequence} 3' \n")
        f.write(f" {''.join(['|'for c in range(len(self.sequence))])}\n")
        f.write(f"3' {comp_seq} 5' \n")
        f.write(f'Reverse Complementary Sequence: {rev_comp_seq}\n\n')
        palindromes = self.id_palindromes()
        f.write(f"Palindromic Sequences in DNA sequence: {palindromes}\n")

        #Melting Temperature of DNA
        tm = self.melting_temperature()
        f.write(f"Melting Temperature:{tm:.2f} °C \n")


        #Protein Translations
        translation = Translation(self.sequence)
        frames = translation.translate_all_frames()
        f.write('Protein Translation Frames:\n')

        #Protein Weights
        annot_prot = translation.annotate_protein()
        f.write(f"Molecular Weights of Proteins:{annot_prot}\n")

        for f_name, prot in frames.items():
          f.write(f"{f_name}: {prot}\n")
          f.write("\n")
        #ORF Positions and Sequences
        orf_positions = self.orf_pos_finder()
        orf_sequences = self.orf_seqs()
        f.write("ORF Positions and Sequences:\n")
        for p, seq in zip(orf_positions, orf_sequences):
          f.write(f"Position: {p}, Sequence: {seq}\n")
        f.write("\n")

        #Primer Design
        primers = self.design_primers()
        f_primers = self.filter_primers(primers)
        f.write("Initial Primer Candidates:\n")
        for x in primers:
          f.write(f"{x}\n")
        f.write("Filtered Primers:\n")
        for y in f_primers:
         f.write(f"{y}\n")
        f.write("\n")

        #K-mer Count
        k = 3
        k_mers = self.count_kmers(k)
        f.write("K-mer Count:\n")
        for kmer, count in k_mers.items():
          f.write(f"{kmer}: {count}\n")

    def plot_kmer_distribution(self, k=3):
      ''' Function creates a barplot to display kmer '''
      k = 3
      k_mers = self.count_kmers(k) #Count the frequency of each k-mer in the sequence
      kdf = pd.DataFrame(list(k_mers.items()), columns=['Kmer', 'Frequency'])
      plt.figure(figsize=(10, 6))
      sns.barplot(x="Kmer", y="Frequency", data=kdf)
      plt.title(f'{k}mer Frequency Distribution')
      plt.xticks(rotation=90)
      plt.show()



    def orf_pos_finder(self):
        ''' Function checks for position of open reading frame of the input sequence '''
        orfs = []
        i = 0
        while i < len(self.sequence) - 2:
          if self.sequence[i:i+3] in self.start_codons: #Record start position
            start = i #Record start position
            i += 3
            for j in range(i , len(self.sequence)-2, 3 ):
              if self.sequence[j:j+3] in self.stop_codons: #Check for stop codon
                end = j + 2 #Record end position
                if end - start > 100: #Check ORF length
                  orfs.append((start, end)) #Append ORF position
                  break
          else:
            i += 3
        return orfs

    def orf_visualizer(self):
      ''' Function creates an interactive open reading frame visualizer'''
      orfs_pos = self.orf_pos_finder() #Find positions of ORFs in the sequence
      fig = go.Figure() #Create a new figure for visualization

      for idx, (start, end) in enumerate(orfs_pos, 1): #Iterate over each ORF position to add it to the figure
        orf_length = end - start
        if orf_length > 100: #Only add ORFs longer than 100 bases to the figure
          fig.add_trace(go.Scatter(x=[start, end], y=[1, 1], mode='lines+markers+text', line=dict(color='tomato', width=4),
            text=f"ORF{idx}", textposition="bottom center", hoverinfo='text',
            hovertext=f"<b>ORF{idx}</b><br>Start: {start}<br>End: {end}<br>Length: {orf_length} bases",
            marker=dict(color='tomato', size=10, opacity=0), showlegend=False))

          #Update layout to improve visualization
      fig.update_layout(title='Visualization of ORFs', xaxis_title='Position in Sequence', yaxis_title='Track',
                      yaxis=dict(showticklabels=False, range=[0.8, 1.2]),  #Adjust y-axis to prevent cutting off text
        plot_bgcolor='white', xaxis=dict(showgrid=True, gridcolor='LightGrey', zeroline=False, rangeslider=dict(visible=True),type='linear'))
      fig.update_layout(width=1200, height=400) #Set figure dimensions
      fig.show()   #Display the figure


class ComplementarySequence(DNASequence):
    def get_complementary_sequence(self):
        ''' Function returns the complementary sequence of input sequence '''
        #Mapping for standard bases plus 'N'
        complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'N': 'N'}
        #Default to 'N' if unexpected character
        return ''.join(complement.get(base, 'N') for base in self.sequence)

    def get_reverse_complement(self):
      ''' Function returns the reverse complement of the input sequence '''
      return self.get_complementary_sequence()[::-1]


class Translation(DNASequence):
    def __init__(self, sequence):
        """Function initializes the Translation object with the input sequence and codon table."""
        super().__init__(sequence)
        self.codon_table = {"GCT": "A", "GCC": "A", "GCA": "A", "GCG": "A",
            "TGT": "C", "TGC": "C","GAT": "D", "GAC": "D","GAA": "E", "GAG": "E",
            "TTT": "F", "TTC": "F","GGT": "G", "GGC": "G", "GGA": "G", "GGG": "G",
            "CAT": "H", "CAC": "H","ATA": "I", "ATT": "I", "ATC": "I",
            "AAA": "K", "AAG": "K","TTA": "L", "TTG": "L", "CTT": "L", "CTC": "L", "CTA": "L", "CTG": "L",
            "ATG": "M","AAT": "N", "AAC": "N","CCT": "P", "CCC": "P", "CCA": "P", "CCG": "P",
            "CAA": "Q", "CAG": "Q","CGT": "R", "CGC": "R", "CGA": "R", "CGG": "R", "AGA": "R", "AGG": "R",
            "TCT": "S", "TCC": "S", "TCA": "S", "TCG": "S", "AGT": "S", "AGC": "S",
            "ACT": "T", "ACC": "T", "ACA": "T", "ACG": "T",
            "GTT": "V", "GTC": "V", "GTA": "V", "GTG": "V",
            "TGG": "W","TAT": "Y", "TAC": "Y", "TAA": "_", "TAG": "_", "TGA": "_"}

    def translate_frame(self, frame):
        ''' Function translates the input sequence '''
        protein_sequence = Seq(self.sequence[frame:]).translate(to_stop=True) #Translate the sequence taking care of the frame shift
        return str(protein_sequence)  #Convert the protein sequence to a string and return it


    def translate_all_frames(self):
        ''' Function translates all the frames of given sequence'''
        frames = {}
        #3 forward frames
        for frame in range(3):
          frames[f"Forward Frame {frame+1}"] = self.translate_frame(frame)
        #3 reverse frames
        reverse_seq = ComplementarySequence(self.sequence).get_reverse_complement()
        for frame in range(3):
          frames[f"Reverse Frame {frame+1}"] = Translation(reverse_seq).translate_frame(frame)
        return frames

    def calc_mol_weight(self, protein_sequence):
      ''' Function computes the moleecular weight of input protein'''
      #Calculate the molecular weight using BioPython
      return molecular_weight(protein_sequence, seq_type='protein')

    def annotate_protein(self):
      ''' Function returns a dictionary of molecular weights of proteins translated from input sequence '''
      all_frames = self.translate_all_frames()
      annotated_frames = {}
      for frame, seq in all_frames.items():
          if seq:  #Ensure sequence is not empty
            mw = self.calc_mol_weight(seq)
            annotated_frames[frame] = {'Sequence': seq, 'Molecular Weight': mw}
      return annotated_frames


class EntrezData:
    def __init__(self, email):
        Entrez.email = email

    def fetch_sequence(self, accession):
      ''' Function fetches fasta sequence associated with accesion id '''
      with Entrez.efetch(db="nucleotide", rettype="fasta", retmode="text", id=accession) as handle:
          #Read the FASTA record
          seq_record = SeqIO.read(handle, "fasta")
          #Extract the sequence from the record
          sequence = str(seq_record.seq)
          #print("Sequence obtained from Entrez:", sequence)
          print("Length of sequence obtained from Entrez:", len(sequence))
      return DNASequence(sequence)


def main():
    ''' Main function that computes everything '''
    #Prompt the user to enter their email address for Entrez
    email = input("Please enter your email address (for Entrez): ")
    entrez_data = EntrezData(email)

    #Prompt the user to enter accession numbers separated by commas
    accession_input = input("Please enter accession numbers separated by commas: ")
    accession_numbers = [acc.strip() for acc in accession_input.split(',')]

    #Loop through each accession number to fetch sequences and process them
    for index, accession in enumerate(accession_numbers):
        try:
            sequence = entrez_data.fetch_sequence(accession) #Fetch the DNA sequence using the accession number
            sequence.label = accession  #Label each sequence by its accession number
            sequence.save_results() #Save the results to a file
            print(f"Results saved for accession {accession}")

            #Visual separator for console output
            print("#" * 20 + " Next Accession Number " + "#" * 20 + "\n")

            #Plot the k-mer distribution for the sequence
            sequence.plot_kmer_distribution()
            print("###############################################################")

            #Visualize the open reading frames (ORFs) for the sequence
            sequence.orf_visualizer()


            #Visual separator for console output if there are more accession numbers to process
            if index < len(accession_numbers) - 1:
                print("#" * 20 + " Next Accession Number " + "#" * 20 + "\n")
        except Exception as e:
            #Handle any errors that occur during sequence fetching and processing
            print(f"Error in fetching sequence for accession id number {accession}: {e}")


if __name__ == '__main__':
    main()

KeyboardInterrupt: Interrupted by user